<a href="https://colab.research.google.com/github/KKAARRIIMM15/Audio-Processing/blob/main/Bioinformatics/Lung_Diagnose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## the Author Email is **karimeldeeb2001@gmail.com**
## the Author what's up number is **+201555604511**

**This project shows how to build and train biLSTM neural network to classify the extracted MFCCs from the audio signal.**


 **Project Idea**

project idea is to scan the the breathing sound of the lung then analyze the audio signal to detect 7 Lung Diseases whether it is (Pneumonia, Bronchiolitis, LRTI, Asthma, URTI, COPD, Normal)


### The dataset contains audio records of 7 Lung Diseases

the dataset is from kaggle [link dataset](https://www.kaggle.com/datasets/vbookshelf/respiratory-sound-database)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import librosa
import numpy as np

In [ ]:

def MFCC( y, sr ):

  MFCC = librosa.feature.mfcc( y=y, sr=sr )

  positiveCoefficient  = np.where(MFCC > 0, MFCC, 0)
  negativeCoefficient  = np.where(MFCC < 0, MFCC, 0)

  pstv_parts = np.array_split(positiveCoefficient.flatten(), 500 )
  ngtv_parts = np.array_split(negativeCoefficient.flatten(), 500 )

  pstv = [ ]
  ngtv = [ ]

  i = 0
  while( i < len(pstv_parts) ):

      if np.any(pstv_parts[i]):
        pstv.append( pstv_parts[i].mean() / 270 )   # Normalize

      else:
        mean_value = 0.0
        pstv.append( mean_value )

      if np.any(ngtv_parts[i]):
        ngtv.append( ngtv_parts[i].mean() / 400)    # Normalize

      else:
        mean_value = 0.0
        ngtv.append( mean_value )

      i = i + 1

  pstv = np.array(pstv)
  ngtv = np.array(ngtv)


  concatenated_arr = np.concatenate(( pstv , ngtv ))
  concatenated_arr = np.round(concatenated_arr , decimals = 5 )

  return concatenated_arr.reshape(10, 100)



(10, 100)

**The reasons why i extracted the MFCCs from the audio signal**
Lung diseases (like asthma, COPD, or pneumonia) often change the way a person breathes or coughs. These changes affect the sound of their breath or cough. Here’s why MFCCs are useful:

**1- Capture Disease-Specific Patterns:** MFCCs can highlight subtle differences in sound that are specific to certain lung diseases. For example, a wheezing sound (common in asthma) will have different MFCCs compared to a normal breath.

**2-Reduce Complexity:** Instead of analyzing the entire raw audio signal (which is huge and noisy), MFCCs give you a smaller, cleaner set of features to work with. This makes it easier for your LSTM model to learn and classify.

**3- Captures Temporal and Spectral Features:** Why It Matters: Lung disease sounds (like coughs or wheezes) have both temporal (how the sound changes over time) and spectral (the frequencies present in the sound) characteristics. MFCCs capture both types of features.

Benefit: This gives your biLSTM model a richer set of information to work with, improving its ability to classify diseases accurately.


In [ ]:
import warnings

Pneumonia_Feature = [ ];        Pneumonia_Label = [ ]

Bronchiolitis_Feature = [ ];    Bronchiolitis_Label = [ ]

LRTI_Feature = [ ];             LRTI_Label = [ ]
Asthma_Feature = [ ];           Asthma_Label = [ ]
URTI_Feature = [ ];             URTI_Label = [ ]

COPD_Feature = [ ];             COPD_Label = [ ]
Healthy_Feature = [ ];          Healthy_Label = [ ]


import pandas as pd
data = pd.read_csv("drive//MyDrive//Respiratory//patient_diagnosis.csv", header=None)


import os
directory = "drive//MyDrive//Respiratory//audio_and_txt_files"
audio_files = os.listdir(directory)

audio_files = [file for file in audio_files if file.endswith('.wav')]


**I labeled the 7 Lung Diseases from 0 to 6**

In [ ]:
i = 0
while(i < len(audio_files) ):

  with warnings.catch_warnings():
    warnings.filterwarnings("error", category=RuntimeWarning)

  y, sr = librosa.load("drive//MyDrive//Respiratory//audio_and_txt_files//" + audio_files[i] )
  try:
    concatenated_arr = MFCC( y , sr )

    if ( concatenated_arr is not None ):
      result = data[ data[0] == int(audio_files[i].split("_")[0] ) ]

      if( result[1].values[0] == "Pneumonia" ):
        Pneumonia_Feature.append( concatenated_arr )
        Pneumonia_Label.append(0)
        print( "Pneumonia" + "   " + str(i) )

      elif( result[1].values[0] == "Bronchiolitis" ):
        Bronchiolitis_Feature.append( concatenated_arr )
        Bronchiolitis_Label.append(1)
        print( "Bronchiolitis" + "   " + str(i) )

      elif( result[1].values[0] == "Healthy" ):
        Healthy_Feature.append( concatenated_arr )
        Healthy_Label.append(2)
        print( "Healthy" + "   " + str(i) )


      elif( result[1].values[0] == "COPD" ):
        COPD_Feature.append( concatenated_arr )
        COPD_Label.append(3)
        #print( "COPD" + "   " + str(i) )


      elif( result[1].values[0] == "URTI" ):
        URTI_Feature.append( concatenated_arr )
        URTI_Label.append(4)
        print( "URTI" + "   " + str(i) )


      elif( result[1].values[0] == "Asthma" ):
        Asthma_Feature.append( concatenated_arr )
        Asthma_Label.append(5)
        print( "Asthma" + "   " + str(i) )


      elif( result[1].values[0] == "LRTI" ):
        LRTI_Feature.append( concatenated_arr )
        LRTI_Label.append(6)
        print( "LRTI" + "   " + str(i) )

  except Exception as e:{}

  i = i + 1


In [ ]:
from sklearn.model_selection import train_test_split

voice = np.concatenate(( Pneumonia_Feature , Bronchiolitis_Feature , Healthy_Feature ,COPD_Feature , URTI_Feature,Asthma_Feature,LRTI_Feature   ), axis=0)
cls = np.concatenate(( Pneumonia_Label , Bronchiolitis_Label , Healthy_Label  , COPD_Label ,URTI_Label , Asthma_Label , LRTI_Label   ), axis=0)

xTrain, xval, yTrain, yval = train_test_split(voice, cls , test_size=0.3, random_state=42)  # 30% for Testing

**Build biLSTM neural network**

In [ ]:
from tensorflow import Bidirectional , LSTM , Dense , Adam , models

model = models.Sequential()
model.add(Bidirectional( LSTM(units=32 , input_shape=(10, 100)  , activation='tanh', recurrent_activation='sigmoid') ) )
model.add(Dense(16, activation="selu" ))
model.add(Dense(units=7, activation='softmax'))



In [ ]:
optimiz = Adam(learning_rate=0.001)

model.compile(optimizer= optimiz , loss='SparseCategoricalCrossentropy', metrics=['accuracy'])   # Compile the model
model.build(input_shape=(None,10, 100))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_7 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dense_14 (Dense)            (None, 16)                1040      
                                                                 
 dense_15 (Dense)            (None, 7)                 119       
                                                                 
Total params: 35207 (137.53 KB)
Trainable params: 35207 (137.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Start Training**

In [ ]:
history = model.fit( xTrain , yTrain , epochs=30, batch_size=10 , validation_data=(xval, yval)  )

Epoch 1/30
81/81 [==============================] - 8s 25ms/step - loss: 0.7435 - accuracy: 0.8672 - val_loss: 0.5245 - val_accuracy: 0.8848
Epoch 2/30
81/81 [==============================] - 1s 11ms/step - loss: 0.5075 - accuracy: 0.8896 - val_loss: 0.5048 - val_accuracy: 0.8848
Epoch 3/30
81/81 [==============================] - 1s 11ms/step - loss: 0.4726 - accuracy: 0.8896 - val_loss: 0.4870 - val_accuracy: 0.8848
Epoch 4/30
81/81 [==============================] - 1s 11ms/step - loss: 0.4462 - accuracy: 0.8896 - val_loss: 0.4543 - val_accuracy: 0.8848
Epoch 5/30
81/81 [==============================] - 1s 10ms/step - loss: 0.4134 - accuracy: 0.8896 - val_loss: 0.4072 - val_accuracy: 0.8848
Epoch 6/30
81/81 [==============================] - 1s 11ms/step - loss: 0.4003 - accuracy: 0.8896 - val_loss: 0.4255 - val_accuracy: 0.8865
Epoch 7/30
81/81 [==============================] - 1s 11ms/step - loss: 0.3921 - accuracy: 0.8896 - val_loss: 0.3898 - val_accuracy: 0.8848
Epoch 8/30
81

In [ ]:
optimiz = Adam(learning_rate=0.0009)
model.compile(optimizer= optimiz , loss='SparseCategoricalCrossentropy', metrics=['accuracy'])   # Compile the model

In [ ]:
history = model.fit( xTrain , yTrain , epochs=40, batch_size=5 , validation_data=(xval, yval)  )

Epoch 1/40
162/162 [==============================] - 6s 14ms/step - loss: 0.2882 - accuracy: 0.9057 - val_loss: 0.2457 - val_accuracy: 0.9174
Epoch 2/40
162/162 [==============================] - 1s 9ms/step - loss: 0.2868 - accuracy: 0.9020 - val_loss: 0.2569 - val_accuracy: 0.9174
Epoch 3/40
162/162 [==============================] - 2s 14ms/step - loss: 0.2677 - accuracy: 0.9156 - val_loss: 0.2263 - val_accuracy: 0.9298
Epoch 4/40
162/162 [==============================] - 2s 14ms/step - loss: 0.2868 - accuracy: 0.9094 - val_loss: 0.2445 - val_accuracy: 0.9215
Epoch 5/40
162/162 [==============================] - 1s 9ms/step - loss: 0.2637 - accuracy: 0.9119 - val_loss: 0.2466 - val_accuracy: 0.9174
Epoch 6/40
162/162 [==============================] - 1s 9ms/step - loss: 0.2698 - accuracy: 0.9169 - val_loss: 0.2793 - val_accuracy: 0.9174
Epoch 7/40
162/162 [==============================] - 1s 9ms/step - loss: 0.2598 - accuracy: 0.9107 - val_loss: 0.2666 - val_accuracy: 0.9215
Epo

In [ ]:
model.save("Lung7.h5")